In [1]:
import pandas as pd

In [2]:
molecules_train = pd.read_json("gated-graph-neural-network-pytorch/molecules_trainFull.json")
molecules_valid = pd.read_json("gated-graph-neural-network-pytorch/molecules_validFull.json")

In [3]:
molecules_train.head()

,graph,node_features,targets
0,"[[0, 1, 1], [1, 2, 2], [1, 1, 3], [3, 1, 4], [...","[[0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",[[-0.4143944162]]
1,"[[0, 1, 1], [1, 1, 2], [2, 1, 3], [3, 1, 4], [...","[[0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0...",[[-0.7727865908]]
2,"[[0, 1, 1], [1, 1, 2], [2, 1, 3], [3, 1, 4], [...","[[0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0...",[[-0.5899948140000001]]
3,"[[0, 1, 1], [1, 1, 2], [2, 1, 3], [3, 1, 4], [...","[[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0...",[[-1.2355191359]]
4,"[[0, 1, 1], [1, 1, 2], [2, 1, 3], [3, 1, 4], [...","[[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0...",[[-1.2417952483999999]]


In [4]:
len(molecules_train), len(molecules_valid)

(120803, 13082)

In [22]:
molecules_train=molecules_train[:1000]
molecules_valid=molecules_valid[:100]

In [28]:
molecules_valid.to_json("gated-graph-neural-network-pytorch/molecules_valid.json", orient="records")

In [29]:
molecules_train.to_json("gated-graph-neural-network-pytorch/molecules_train.json", orient="records")